In [80]:
L_max = 365 # upper bound period - we have in total 365 total possible events.
import numpy as np
import random

class RandomGenerator():
    def __init__(self, max_value=100):
        self.max_value = max_value
    def nextInt(self):
        return random.randint(1, self.max_value)
    

In [81]:
class Record():
    def __init__(self, time, value):
        self.time = time
        self.val = value

In [177]:
# make fake data
from collections import deque
d = [1 if i % 6==0 or i % 30==0 else -1 for i in range(1, 365)]
data_d = deque(d)
data_d.rotate(3) # skew the data a bit (add offset)
data = list(data_d)

# add data to records
records = list()
for t, value in enumerate(data):
    record = Record(t, value)
    records.append(record)

In [178]:
class CoverageBase(object):
    def __init__(self, records, L_max, alpha):
        self.records = records
        
        self.rand = RandomGenerator(100) # random int generator
        self.T_cnt = 0 # positive observations
        self.F_cnt = 0 # negative observations
        
        self.L_max = L_max
        self.N = len(records)
        self.alpha = alpha
        
        self.O = np.zeros(self.N) # observation sequence (transformed from raw records)
        
        for record in self.records:
            t = record.time
            if record.val == 1:
                self.O[t] = 1
                self.T_cnt += 1
            elif record.val == -1:
                self.O[t] = -1
                self.F_cnt += 1

    def getPositiveSlotScore(self, H, L):
        slot_score = dict()
        
        l_tp = np.empty(L)
        l_fp = np.empty(L)
        
        for i in range(self.N):
            if H[i] == 1:
                l_tp[i%L] += 1
            elif H[i] == -1:
                l_fp[i%L] += 1
        
        for l in range(L):
            score = 1-self.alpha * l_tp[l] / self.T_cnt - self.alpha * l_fp[l] / self.F_cnt + self.alpha
            if score > self.alpha:
                slot = L + "\t" + l
                slot_score[slot] = score
        
        return slot_score

In [228]:
class CoverageEfficient(CoverageBase):
    def __init__(self, records, L_max, alpha):
        super().__init__(records, L_max, alpha)
    
    def initialSScores(self):
        self.P = np.zeros(shape=(self.N))
        # initialize 2d arrays
        self.S_tp = np.zeros(shape=(L_max,L_max))
        self.S_fp = np.zeros(shape=(L_max,L_max))
        self.S_valid = np.full((L_max, L_max), True)
        self.S_val = np.zeros(shape=(L_max,L_max))
        
        for L in range(2, self.L_max):
            for i in range(self.N):
                if self.O[i] == 1:
                    self.S_tp[L][i % L] += 1
                elif self.O[i] == -1:
                    self.S_fp[L][i % L] += 1
                    
            for l in range(L):
                score = (1 - self.alpha) * self.S_tp[L][l] / self.T_cnt - self.alpha * self.S_fp[L][l] / self.F_cnt + self.alpha
                if score > self.alpha:
                    self.S_val[L][l] = score
        
    def top1S(self):
        score_max = 0.0
        S = ""
        for L in range(2, self.L_max):
            for l in range(L):
                print(self.S_val[L][l])
                if(self.S_val[L][l] == False):
                    continue
                if self.S_val[L][l] > score_max:
                    score_max = self.S_val[L][l]
                    S = "{}\t{}".format(L, 1)
        return S
    
    def updateS_tpfp_val(self, L:int, l:int):
        S_changed = set()
        
        for i in range(1, self.N, L):
            if self.P[i] == 0 and self.O[i] != 0:
                for L_tmp in range(1, self.L_max):
                    l_tmp = i % L_tmp
                    if self.S_valid[L_tmp][l_tmp] == False:
                        continue
                    if self.O[i] > 0:
                        self.S_tp[L_tmp][l_tmp] -= 1
                    elif self.O[i] < 0:
                        self.S_fp[L_tmp][l_tmp] -= 1
                    S_changed.add("{}\t{}".format(L_tmp, l_tmp))
                self.P[i] = 1
        
        for S in S_changed:
            L_tmp, l_tmp = list(map(int, S.split("\t")))
            score = (1 - self.alpha) * self.S_tp[L_tmp][l_tmp] / self.T_cnt - self.alpha * self.S_fp[L_tmp][l_tmp] / self.F_cnt + self.alpha
            if score > self.alpha:
                self.S_val[L_tmp][l_tmp] = score
            else:
                self.S_val[L_tmp][l_tmp] = 0
                
        for L_tmp in range(L, self.L_max, L):
            for l_tmp in range(l, L_tmp, L):
                self.S_valid[L_tmp][l_tmp] = False
        
    def findL
ce = CoverageEfficient(records, 365, 0.6)
ce.initialSScores()
ce.updateS_tpfp_val(10,3)


In [191]:
rand = RandomGenerator(100)
O = list() # observation sequence (transformed from raw records)
T_cnt = 0 # positive observations
F_cnt = 0 # negative observations
N = 0
